In [1]:
import torch
import pandas as pd
import catboost as cb
from sentence_transformers import SentenceTransformer
import numpy as np
import gensim.downloader as api
from skmultilearn.problem_transform import ClassifierChain
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, hamming_loss
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from preprocessing.utils import is_sentence_in_boundaries
from datasets_utils import get_luxury_data, get_tech_data, get_retail_data, get_big_basket_data
from preprocess import preprocess, with_category_features

device = 'cuda' if torch.cuda.is_available() else 'cpu'

[nltk_data] Downloading package stopwords to /home/stepan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/stepan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/stepan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
device

'cuda'

In [3]:
big_basket_data = get_big_basket_data()
# luxury_data = get_luxury_data()
# tech_data = get_tech_data()
# retail_data = get_retail_data()

datasets = [big_basket_data]
dataset_names = ['Big basket']
# datasets = [big_basket_data, retail_data, luxury_data, tech_data]
# dataset_names = ['Big basket', 'Retail', 'Luxury', 'Tech']

# Get datasets with description column preprocessed
big_basket_data['description'] = big_basket_data['description'].apply(preprocess)
# tech_data['description'] = tech_data['description'].apply(preprocess)
# luxury_data['description'] = luxury_data['description'].apply(preprocess)
# retail_data['description'] = retail_data['description'].apply(preprocess)

# Preprocess categories
big_basket_data = with_category_features(big_basket_data)
# tech_data = with_category_features(tech_data)
# luxury_data = with_category_features(luxury_data)
# retail_data = with_category_features(retail_data)

big_basket_data = big_basket_data[big_basket_data['description'].apply(lambda x: is_sentence_in_boundaries(x, max_tokens=200))]
# retail_data = retail_data[retail_data['description'].apply(lambda x: is_sentence_in_boundaries(x, max_tokens=250))]
# luxury_data = luxury_data[luxury_data['description'].apply(lambda x: is_sentence_in_boundaries(x, max_tokens=100))]
# tech_data = tech_data[tech_data['description'].apply(lambda x: is_sentence_in_boundaries(x, max_tokens=200))]

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/src/preprocess.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[category] = data['category'].apply(lambda x: 1 if category in x else 0)
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/src/preprocess.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[category] = data['category'].apply(lambda x: 1 if category in x else 0)
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/src/preprocess.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is us

In [4]:
big_basket_X_train, big_basket_X_test, big_basket_y_train, big_basket_y_test = train_test_split(
    big_basket_data['description'],
    big_basket_data[[column for column in big_basket_data.columns if column != 'description']],
    test_size=0.2,
    random_state=13
)

In [5]:
big_basket_X_train.shape, big_basket_X_test.shape

((21156,), (5289,))

### TF-IDF

In [6]:
def tfidf_vectorize(train_data, test_data):
    tfidf = TfidfVectorizer()
    tfidf_train_data = tfidf.fit_transform(train_data)
    tfidf_test_data = tfidf.transform(test_data)
    return tfidf_train_data, tfidf_test_data

In [7]:
big_basket_X_train_tfidf, big_basket_X_test_tfidf = tfidf_vectorize(
    train_data=big_basket_X_train,
    test_data=big_basket_X_test,
)

In [11]:
big_basket_tfidf_model = GridSearchCV(
    ClassifierChain(classifier=DecisionTreeClassifier(criterion='log_loss')),
    param_grid=[
        {
            'classifier': [MultinomialNB()],
            'classifier__alpha': [0.7],
        },
        {
            'classifier': [DecisionTreeClassifier()],
            'classifier__criterion': ['log_loss'],
        },
    ],
    n_jobs=8,
)

big_basket_tfidf_model.fit(big_basket_X_train_tfidf, big_basket_y_train.to_numpy())

GridSearchCV(estimator=ClassifierChain(classifier=DecisionTreeClassifier(criterion='log_loss'),
                                       require_dense=[True, True]),
             n_jobs=8,
             param_grid=[{'classifier': [MultinomialNB()],
                          'classifier__alpha': [0.7]},
                         {'classifier': [DecisionTreeClassifier()],
                          'classifier__criterion': ['log_loss']}])

In [10]:
# big_basket_tfidf_model = ClassifierChain(
#     classifier=DecisionTreeClassifier(criterion='log_loss'),
# )

# big_basket_tfidf_model.fit(big_basket_X_train_tfidf, big_basket_y_train.to_numpy())

In [12]:
big_basket_tfidf_model.best_params_, big_basket_tfidf_model.best_score_

({'classifier': DecisionTreeClassifier(), 'classifier__criterion': 'log_loss'},
 0.619729925712593)

In [13]:
big_basket_y_pred_tfidf = big_basket_tfidf_model.predict(big_basket_X_test_tfidf)

In [14]:
print(f"Accuracy: {accuracy_score(big_basket_y_test.to_numpy(), big_basket_y_pred_tfidf)}")
print(f"Precision (macro): {precision_score(big_basket_y_test.to_numpy(), big_basket_y_pred_tfidf, average='macro')}")
print(f"Precision (micro): {precision_score(big_basket_y_test.to_numpy(), big_basket_y_pred_tfidf, average='micro')}")
print(f"Recall (macro): {recall_score(big_basket_y_test.to_numpy(), big_basket_y_pred_tfidf, average='macro')}")
print(f"Recall (micro): {recall_score(big_basket_y_test.to_numpy(), big_basket_y_pred_tfidf, average='micro')}")
print(f"Hamming loss: {hamming_loss(big_basket_y_test.to_numpy(), big_basket_y_pred_tfidf)}")

Accuracy: 0.637738702968425
Precision (macro): 0.651528745488174
Precision (micro): 0.7623174052983411
Recall (macro): 0.6352327537863509
Recall (micro): 0.7692371752165224
Hamming loss: 0.010273499425513038


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Word2Vec

In [16]:
wv = api.load('word2vec-google-news-300')

In [17]:
def to_w2v_embedding(sentence):
    embeddings = []

    for word in sentence.split():
        if word in wv:
            embeddings.append(wv[word])
    embeddings = np.array(embeddings)
    return np.mean(embeddings, axis=0)

In [18]:
big_basket_X_train_w2v = np.array([x for x in big_basket_X_train.apply(to_w2v_embedding)])
big_basket_X_test_w2v = np.array([x for x in big_basket_X_test.apply(to_w2v_embedding)])

In [19]:
big_basket_w2v_model = GridSearchCV(
    ClassifierChain(classifier=DecisionTreeClassifier(criterion='log_loss')),
    param_grid=[
        {
            'classifier': [MultinomialNB()],
            'classifier__alpha': [0.7],
        },
        {
            'classifier': [DecisionTreeClassifier()],
            'classifier__criterion': ['log_loss'],
        },
    ],
    n_jobs=8,
)

big_basket_w2v_model.fit(big_basket_X_train_w2v, big_basket_y_train.to_numpy())

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
5 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/skmultilearn/problem_transform/cc.py", line 154, in fit
    self.classifiers_[label] = self.classifier.fit(self._ensure_input_format(
  File "/h

GridSearchCV(estimator=ClassifierChain(classifier=DecisionTreeClassifier(criterion='log_loss'),
                                       require_dense=[True, True]),
             n_jobs=8,
             param_grid=[{'classifier': [MultinomialNB()],
                          'classifier__alpha': [0.7]},
                         {'classifier': [DecisionTreeClassifier()],
                          'classifier__criterion': ['log_loss']}])

In [20]:
big_basket_w2v_model.best_params_, big_basket_w2v_model.best_score_

({'classifier': DecisionTreeClassifier(), 'classifier__criterion': 'log_loss'},
 0.46265871577996415)

In [21]:
big_basket_y_pred_w2v = big_basket_w2v_model.predict(big_basket_X_test_w2v)

In [22]:
print(f"Accuracy: {accuracy_score(big_basket_y_test.to_numpy(), big_basket_y_pred_w2v)}")
print(f"Precision (macro): {precision_score(big_basket_y_test.to_numpy(), big_basket_y_pred_w2v, average='macro')}")
print(f"Precision (micro): {precision_score(big_basket_y_test.to_numpy(), big_basket_y_pred_w2v, average='micro')}")
print(f"Recall (macro): {recall_score(big_basket_y_test.to_numpy(), big_basket_y_pred_w2v, average='macro')}")
print(f"Recall (micro): {recall_score(big_basket_y_test.to_numpy(), big_basket_y_pred_w2v, average='micro')}")
print(f"Hamming loss: {hamming_loss(big_basket_y_test.to_numpy(), big_basket_y_pred_w2v)}")

Accuracy: 0.47872943845717525
Precision (macro): 0.46211925380095104
Precision (micro): 0.6238606114382512
Recall (macro): 0.4818617674637915
Recall (micro): 0.644070619586942
Hamming loss: 0.016247436624634584


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### BERT embeddings

In [23]:
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2').to(device)

In [30]:
big_basket_X_train_emb = sbert_model.encode(big_basket_X_train.values.tolist())
big_basket_X_test_emb = sbert_model.encode(big_basket_X_test.values.tolist())

In [35]:
big_basket_X_train_emb.shape, big_basket_X_test_emb.shape

((21156, 384), (5289, 384))

In [36]:
big_basket_emb_model = GridSearchCV(
    ClassifierChain(classifier=DecisionTreeClassifier(criterion='log_loss')),
    param_grid=[
        {
            'classifier': [MultinomialNB()],
            'classifier__alpha': [0.7],
        },
        {
            'classifier': [DecisionTreeClassifier()],
            'classifier__criterion': ['log_loss'],
        },
    ],
    n_jobs=8,
)

big_basket_emb_model.fit(big_basket_X_train_emb, big_basket_y_train.to_numpy())

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

GridSearchCV(estimator=ClassifierChain(classifier=DecisionTreeClassifier(criterion='log_loss'),
                                       require_dense=[True, True]),
             n_jobs=8,
             param_grid=[{'classifier': [MultinomialNB()],
                          'classifier__alpha': [0.7]},
                         {'classifier': [DecisionTreeClassifier()],
                          'classifier__criterion': ['log_loss']}])

In [37]:
big_basket_y_pred_emb = big_basket_emb_model.predict(big_basket_X_test_emb)

In [38]:
print(f"Accuracy: {accuracy_score(big_basket_y_test.to_numpy(), big_basket_y_pred_emb)}")
print(f"Precision (macro): {precision_score(big_basket_y_test.to_numpy(), big_basket_y_pred_emb, average='macro')}")
print(f"Precision (micro): {precision_score(big_basket_y_test.to_numpy(), big_basket_y_pred_emb, average='micro')}")
print(f"Recall (macro): {recall_score(big_basket_y_test.to_numpy(), big_basket_y_pred_emb, average='macro')}")
print(f"Recall (micro): {recall_score(big_basket_y_test.to_numpy(), big_basket_y_pred_emb, average='micro')}")
print(f"Hamming loss: {hamming_loss(big_basket_y_test.to_numpy(), big_basket_y_pred_emb)}")

Accuracy: 0.4437511816978635
Precision (macro): 0.43878720634252805
Precision (micro): 0.5773605484574634
Recall (macro): 0.45847864418845785
Recall (micro): 0.6171718854097269
Hamming loss: 0.018219963058306792


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
